In [1]:
# power = 260mW 
# T = 64 C 
# %matplotlib ipympl 
%matplotlib qt 
%load_ext autoreload 
%autoreload 2
from pianoq.lab.asi_cam import ASICam
from pianoq.lab.dmd import DMD
import numpy as np 
from pianoq.misc.mplt import mimshow, mplot
import datetime 
import time 
import matplotlib.pyplot as plt
from pianoq.lab.time_tagger import QPTimeTagger
from pianoq.lab.photon_scan import PhotonScanner
from pianoq.lab.photon_counter import PhotonCounter
from pianoq.lab.thorlabs_motor import ThorlabsKcubeDC, ThorlabsKcubeStepper
from pianoq.lab.lee_holo.generate_lee import main, make_oam_donut, orthogonal_lee

In [ ]:
# Laser@260mW
dmd = DMD()
dmd.set_white()


In [9]:
integration_time = 1
# coin_window = 4e-9 
ph = PhotonCounter(serial_port='COM6', integration_time=integration_time)
coin_window = 1.2e-9 # s 
# When DMD-plane fiber is 100um 1m. Delays=(0, 500) when both are 50um 2m fibers
# tt = ph = QPTimeTagger(integration_time=integration_time, coin_window=coin_window, single_channel_delays=(0, 5000), remote=True, address='132.64.81.123:41101')  

In [ ]:
# Using TT
all_s1 = []
all_s2 = []
all_c = []
tnow = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
save_to = rf"G:\My Drive\Projects\DMD\Alon results\Quantum\{tnow}_detectors_bug_4_channels_time_tagger1.npz"
for i in range(100000):
    data, data_std, integration_time = ph.read()
    s1, s2, c = tt.read_interesting()
    s1, s2, c = s1.item(), s2.item(), c.item()
    all_s1.append(s1)
    all_s2.append(s2)
    all_c.append(c)
    print(f'{s1=}, {s2=}, {c=}')
    if i % 60 == 0:
        s1np = np.array(all_s1)
        s2np = np.array(all_s2)
        cnp = np.array(all_c)
        np.savez(save_to, s1=s1np, s2=s2np, c=cnp)


In [ ]:
# using ph 
all_data = []
tnow = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
save_to = rf"G:\My Drive\Projects\DMD\Alon results\Quantum\{tnow}_detectors_bug_4_channels_ph_covered.npz"
for i in range(100000):
    data, data_std, integration_time = ph.read()
    all_data.append(data)
    print(data)
    if i % 60 == 0:
        np.savez(save_to, all_data=np.array(all_data))


In [ ]:
# With TT 
data = np.load(r"G:\My Drive\Projects\DMD\Alon results\Quantum\detectors_bug_4_channels_photon_counter1.npz")

fig, ax = plt.subplots()
ax.plot(data['s1'], label='s1')
ax.plot(data['s2'], label='s2')
ax.plot(data['c'], label='c')
ax.legend()
fig.show()


In [6]:
# With ph
 
data = np.load(r"G:\My Drive\Projects\DMD\Alon results\Quantum\detectors bug\2025_11_20_10_19_17_detectors_bug_4_channels_ph_covered.npz")
# data = np.load(r"G:\My Drive\Projects\DMD\Alon results\Quantum\detectors bug\2025_11_20_10_08_51_detectors_bug_4_channels_ph.npz")
data['all_data']
fig, ax = plt.subplots()
for i in range(4):
    ax.plot(data['all_data'][:,i], label=f'ch{i+1}')
ax.set_ylabel('counts per second')
ax.set_xlabel('measurement number (1s each)')
ax.legend()
fig.show()